In [ ]:
import uproot as up
import matplotlib.pyplot as plt
import numpy as np
import awkward as ak
import re
import pandas as pd
from uproot3_methods.classes.TLorentzVector import TLorentzVector
from tqdm.notebook import tqdm
import pandas as pd

In [ ]:
file = up.open('../build/evtOutput.root')
ntp = file["ntp"]

In [ ]:
class rrFpy_pandas:
    def __init__(self, file_name):
        file = up.open('../build/evtOutput.root')
        ntp = file["ntp"]
        df = pd.DataFrame(index = ak.to_pandas(ntp["pz"].array()).index, columns=["px", "py", "pz", "E", "Id"])
        df["px"] = ak.to_pandas(ntp["px"].array())
        df["py"] = ak.to_pandas(ntp["py"].array())
        df["pz"] = ak.to_pandas(ntp["pz"].array())
        df["E"] = ak.to_pandas(ntp["E"].array())
        df = df.reset_index()
        self.df_pivoted = pd.pivot_table(df, index = "entry", columns="subentry", values = ["px","py", "pz","E", "Id"])
        self.df_pivoted["filter"] = True
        file.close()
    def sign(self, x):
        return -1 if x<0 else 1
    def parse_var(self, var):
        [name, numbers] = var.split("_")
        func = {'m2': lambda t, x, y, z: t**2-x**2-y**2-z**2,
                'm': lambda t, x, y, z: np.sqrt(t**2-x**2-y**2-z**2),
                'pt2': lambda t, x, y, z: x**2+y**2,
                 'pt': lambda t, x, y, z: np.sqrt(x**2+y**2)}[name]
        part_list = [int(n) for n in re.sub('([0-9])',r'\1 ',numbers.replace('m','-')).split(' ')[:-1]]
        return func, part_list
    def __getitem__(self, key):
        if type(key) == str:
            df_res = pd.DataFrame(index=self.df_pivoted.index, columns = ["px","py","pz","E", "out"]).fillna(0)
            func, part_list = self.parse_var(key)
            for i in part_list:
                abs_i = abs(i)
                for v in ["px","py","pz","E"]:
                    df_res[v] += self.sign(i)*self.df_pivoted[v,abs_i]
            df_res = df_res[ self.df_pivoted["filter"]]
            return func( df_res["E"], df_res["px"], df_res["py"], df_res["pz"])
        elif type(key) == list:
            return [self[k] for k in key]
    
    def cut(self, condition):
        if (condition == True):
            self.df_pivoted["filter"] = True
        elif type(condition) == str:
            condition = re.sub('([<>=])',r' \1 ', condition)
            var, operation, lhs = [s for s in condition.split(' ') if len(s)>0]
            print([var, operation, lhs])
            if(operation == '>'):
                lhs = float(lhs)
                self.df_pivoted["filter"] = self.df_pivoted["filter"] & (self[var] > lhs) 
            elif operation == '<':
                lhs = float(lhs)
                self.df_pivoted["filter"] = self.df_pivoted["filter"] & (self[var] < lhs) 
            elif operation == '=':
                if '+-' in lhs:
                    lhs, error = [float(n) for n in lhs.split('+-')]
                else:
                    lhs = float(lhs)
                    error = 0
                self.df_pivoted["filter"] = self.df_pivoted["filter"] & \
                    (self[var] > lhs - error) & (self[var] < lhs + error)
        elif type(condition) == list:
            for cond in condition:
                self.cut(cond)
        return np.count_nonzero(self.df_pivoted["filter"])

In [ ]:
rrF = rrFpy_pandas("../build/evtOutput.root")

In [ ]:
rrF.cut(True)
rrF.cut("m_12=1+-0.5")

In [ ]:
rrF["m_12"].plot.hist()

In [ ]:
rrF.df_pivoted["filter"] & (rrF["m_13"] > 1)

In [ ]:
len(rrF.df_pivoted)

In [ ]:
rrF.df_pivoted["filter"] = (rrF["m_23"] > 1)

In [ ]:
rrF["m_23"].plot.hist(bins = 30)

In [ ]:
hh = rrF[ ["m_12","m2_12"]]
hh[0] - np.sqrt(hh[1])